# CAD(2変数バージョン)

このスクリプトでは, CADを次の3つの段階に分けて行う.

- 射影段階: CADを与える多項式族の次数を下げる.
- 底段階:
- 持ち上げ段階:

まずは簡単のために多項式の**変数が2個の場合に限り**実装する.
その場合, 射影段階と持ち上げ段階の操作は1回で済む.

In [4]:
x,y = var("x y")
R = PolynomialRing(PolynomialRing(QQ,"y"),"x")

F = [x^2+y^2-1,x^2-y^3] # ここにCADしたい多項式のリストを渡す

## 射影段階

2変数多項式の族$ F $が与えられたとき, 1変数多項式の族$ F_1 \cup F_2 \cup F_3 $を次のように構成する.

- $ F_1 $: $ f \, (f \in F) $の変数yに対する係数の多項式全体
- $ F_2 $: $ \mathrm{gcd}(f, df/dx) \, (f \in F) $の変数yに対する係数の多項式全体
- $ F_3 $: $ \mathrm{gcd}(f, g) \, (f \neq g \in F) $の変数yに対する係数の多項式全体

In [2]:
# 変数xについてk次以下の部分を取り出す関数
def T(k,f):
    return f.mod(R(x^(k+1)))

In [5]:
def projection(F:list):
    F = [R(f) for f in F]
    G = []
    
    # step1: F_1の計算
    for f in F:
        for term in f.coefficients():
            term = term/term.lc() # 先頭項を1にする(こういうのは2変数でしかダメっぽい)
            if term.degree()>0 and term not in G:
                G.append(term)
    # step2: F_2の計算 
    for f in F:
        for l in range(f.degree()+1):
            for k in range(l, f.degree()+1):
                for i in range(l+1):
                    if T(k,f).diff()==0:
                        continue
                    pscs = [term.leading_coefficient() for term in T(k,f).subresultants(T(k,f).diff())]
                    pscs = [term/term.lc() for term in pscs] # 先頭項を1にする
                    for psc in pscs: 
                        if psc.degree()>0 and psc not in G:
                            G.append(psc)
    # step3: F_3の計算
    for f in F:
        for g in F:
            if f==g:
                continue
            for l in range(min(f.degree(),g.degree())+1):
                for k in range(l,f.degree()+1):
                    for j in range(l, f.degree()+1):
                        if T(k,f)==0 or T(j,g)==0:
                            continue
                        pscs = [term.leading_coefficient() for term in T(k,f).subresultants(T(j,g))]
                        pscs = [term/term.lc() for term in pscs] # 先頭項を1にする
                        for psc in pscs:
                            if psc.degree()>0 and psc not in G:
                                G.append(psc)
    
    return G

G = projection(F);G

[y^2 - 1, y^3, y^6 + 2*y^5 + y^4 - 2*y^3 - 2*y^2 + 1, y^3 + y^2 - 1]

## 底段階(未実装)

射影段階で求めた1変数多項式の族 $ G $ 不変な $ \mathbb{R} $ のセル分割を与える.

## 持ち上げ段階(未実装)

底段階で求めたセル分割を, $ \mathbb{R}^2 $ のセル分割に持ち上げる.

> **有理数でない点での持ち上げ方についてまだ分かっていない.**